# Sequence Classification using Recurrent Neural Networks(RNN)
In this homework, you will learn how to train a recurrent neural network for human action classification. RNN is designed handle sequential data. The network can incorporate both past history and current input. [This](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) is a very good tutorial. You should read it before you start.

## Setup
Please make sure you have h5py and torchnet installed
> pip install h5py

> pip install git+https://github.com/pytorch/tnt.git@master


In [1]:
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt
from torchsummary import summary

use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor


C:\Users\Abhay Goyal\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


use cuda: False


## Dataset
The data we are using is skeleton data, which indicates the 3D locations of body joints. In total, there are 25 body joints. It is collected by Kinect v2. To make it easier, each sequence have same number of frames. You need to classify 10 different actions. There are 2000 training sequences, 400 validation sequences, and 500 test sequences. Each sequence has 15 frames, each frame is a 75-dimension vector (3*25). 

For your convenience, we provide the dataloader for you.


In [2]:
class Dataset(DD.Dataset):
    # subset can be: 'train', 'val', 'test'
    def __init__(self, data_path, subset='train'):
        super(Dataset, self).__init__()
        self.data_path = os.path.join(data_path, '%s_data.h5'%subset)
        self.subset = subset

        with h5py.File(self.data_path) as f:
            self.data = np.array(f['data'])

        if subset != 'test':
            self.label_path = os.path.join(data_path, '%s_label.h5'%subset)
            with h5py.File(self.label_path) as f:
                self.label = np.array(f['label'])

        self.num_sequences = self.data.shape[0]
        self.seq_len = self.data.shape[1]
        self.n_dim = self.data.shape[2]

    def __getitem__(self, index):
        seq = self.data[index]
        if self.subset != 'test':
            label = int(self.label[index])
            sample = {'seq': seq, 'label': label}
        else:
            sample = {'seq': seq}
        return sample

    def __len__(self):
        print(self.num_sequences)
        return self.num_sequences

trSet = Dataset('./data', subset='train')
valSet = Dataset('./data', subset='val')
tstSet = Dataset('./data', subset='test')

batch_size = 100
trLD = DD.DataLoader(trSet, batch_size=batch_size,
       sampler=DD.sampler.RandomSampler(trSet),
       num_workers=0, pin_memory=False)
valLD = DD.DataLoader(valSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(valSet),
       num_workers=0, pin_memory=False)
tstLD = DD.DataLoader(tstSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(tstSet),
       num_workers=0, pin_memory=False)

input_dim = trSet.n_dim
# print(type(trSet))
print(input_dim)
num_class = 10

75


## Model
Pytorch has implemented different types of recurrent layers for you. For this homework, you can use any type of RNNs as you want:
> torch.nn.RNN()

> torch.nn.LSTM()

> torch.nn.GRU()

You can check details for different types of recurrent layers here: [RNN](http://pytorch.org/docs/master/nn.html#torch.nn.RNN), [LSTM]( http://pytorch.org/docs/master/nn.html#torch.nn.LSTM), [GRU](http://pytorch.org/docs/master/nn.html#torch.nn.GRU)


### Implement a specific model
In this section, you need to implement a model for sequence classification. The model has following layers:
* A linear layer that can map features of 75-dimension to 100-dimension.
* 1 Layer LSTM layer with hidden size of 100
* A linear layer that goes from 100 to num_class (10). 

An LSTM layer takes an input of size of (batch_size, seq_len, fea_dim) and outputs a variable of shape (batch_size, seq_len, hidden_size). In this homework, the classification score for a sequence is the classification score for the last step of rnn_outputs.



In [3]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
        ############## 1st To Do (10 points) ##############
        ###################################################
        self.project_layer = nn.Linear(75,100)
        self.recurrent_layer = nn.LSTM(100, 100)
        self.classify_layer = nn.Linear(100, num_class)   
        ###################################################
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        rnn_outputs, (hn, cn) = self.recurrent_layer(self.project_layer(input))
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model = SequenceClassify()
# model = SequenceClassify()
# summary(model,(2000,15,75))

## Train the model
After you have the dataloader and model, you can start training the model. Define a SGD optimizer with learning rate of 1e-3, and a cross-entropy loss function:

In [4]:
################ 2nd To Do  (5 points)##################
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)
criterion = nn.CrossEntropyLoss()

# optimizer = None
# criterion = None

In [5]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence)
        input_label_var = Variable(label)

        # compute output
#         output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data[0])
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 1
evaluate_every_epoch = 5
for e in range(num_epochs):
#     print(len(trLD))
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


2000


C:\Users\Abhay Goyal\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train Epoch: 0  , Loss: 2.3130,  Accuracy: 10.00
400
val Epoch: 0  , Loss: 2.3092,  Accuracy: 9.50


## Submit your results 

### Train a better model for action recognition!
Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves better accuracy on the action recognition validation set. 


### Testing the model and reporting the results
Test the model on the testing set and save the results as a .csv file. 
submit the results.csv file generated by predict_on_test(). Also mention the best performance on the Validation set, and submit the corresponding results csv file which results in the best performance. 
################ 3rd To Do  (15 points) ###############


In [6]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)

In [7]:
import torchvision.models.inception as Inception

class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
        self.convolut = nn.Sequential(
                nn.Conv2d(1, 300, kernel_size = 3, stride=1),
                nn.MaxPool2d(2, stride=2),
                nn.LeakyReLU(negative_slope=0.3),
                nn.BatchNorm2d(300),
                Inception.InceptionA(300,200),
                nn.Conv2d(424, 100, kernel_size=2, stride=1),
                nn.MaxPool2d(kernel_size = 2),
                nn.LeakyReLU(0.3),
                nn.BatchNorm2d(100),
                Inception.InceptionA(100,20),
                Flatten(),
                nn.Linear(8296,100)
        )
        
        self.recurrent_layer = nn.LSTM(100,50,1)
        self.project_layer = nn.Linear(50,10)
        
    def forward(self, input1, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        convolution_input = input1.view(100,1, 15,75)
        convolution_output = self.convolut(convolution_input)
#         print("Convolution output",convolution_output)
        lstm_input = convolution_output.view(100, 1, 100)
        rnn_outputs, (hn, cn) = self.recurrent_layer(lstm_input)
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.project_layer(rnn_outputs[:,-1])
        return logits


model = SequenceClassify()
# summary(model, input_size=trSet.shape)

In [8]:
# optimizer = torch.optim.SGD(model.parameters(), lr = 1e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer = torch.optim.Adadelta(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

In [9]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence)
        input_label_var = Variable(label)

        # compute output
#         output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data[0])
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 50
evaluate_every_epoch = 1
for e in range(num_epochs):
#     print(len(trLD))
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


2000


C:\Users\Abhay Goyal\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train Epoch: 0  , Loss: 2.3048,  Accuracy: 10.50
400
val Epoch: 0  , Loss: 2.3065,  Accuracy: 10.25
2000
train Epoch: 1  , Loss: 2.2964,  Accuracy: 10.00
400
val Epoch: 1  , Loss: 2.3018,  Accuracy: 8.25
2000
train Epoch: 2  , Loss: 2.2893,  Accuracy: 13.20
400
val Epoch: 2  , Loss: 2.2882,  Accuracy: 9.75
2000
train Epoch: 3  , Loss: 2.2829,  Accuracy: 16.65
400
val Epoch: 3  , Loss: 2.2777,  Accuracy: 16.00
2000
train Epoch: 4  , Loss: 2.2737,  Accuracy: 20.35
400
val Epoch: 4  , Loss: 2.2681,  Accuracy: 21.00
2000
train Epoch: 5  , Loss: 2.2645,  Accuracy: 22.75
400
val Epoch: 5  , Loss: 2.2572,  Accuracy: 23.75
2000
train Epoch: 6  , Loss: 2.2507,  Accuracy: 25.10
400
val Epoch: 6  , Loss: 2.2440,  Accuracy: 27.25
2000
train Epoch: 7  , Loss: 2.2338,  Accuracy: 26.45
400
val Epoch: 7  , Loss: 2.2250,  Accuracy: 28.75
2000
train Epoch: 8  , Loss: 2.2161,  Accuracy: 26.55
400
val Epoch: 8  , Loss: 2.2050,  Accuracy: 32.25
2000
train Epoch: 9  , Loss: 2.1926,  Accuracy: 28.85
400
val 

In [ ]:
# Use your best model to generate results on test set and validation set.

# generate csv file for test set
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results1.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i].item())+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(model, valLD)
print(count)

In [48]:
# Use your best model to generate results on test set and validation set.

# generate csv file for test set
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i].item())+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(model, tstLD)
print(count)

500
500


## Report the performance
################ 4th To Do  (5 points)##################

In this cell, you should write an explanation of what you did (network architecture, optimiziter, learning rate, epoches) and any visualizations or graphs that you make in the process of training and evaluating your network.


Network Architecture:
<br>
3x3 Conv2d Layer with 300 filters and stride 1
<br>
MaxPool2d layer
<br>
LeakyReLU layer with slope 0.3
<br>
BatchNorm2d layer
<br>
Inception layer
<br>
2x2 Conv2d layer with 100 filters and stride 1
<br>
MaxPool2d layer
<br>
LeakyReLU layer of slope 0.3
<br>
BatchNorm2d Layer
<br>
Inception Layer
<br>
Flatten Layer
<br>
Linear layer
<br>
LSTM layer
<br>
Linear layer to convert to the 10 classes
<br>
Optimizer
<br>
AdaDelta optimizer with learning rate of 0.01
<br>
CrossEntropy
<br>
50 epochs
<br>